Within a single batch, you must have the same number of timesteps (this is typically where you see 0-padding and masking). But between batches there is no such restriction. During inference, you can have any length.  
(https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras)

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

def rnn_model(num_tokens, num_units):
    # encoder
    enc_input = Input(shape = (None, num_tokens))
    enc_output, enc_state = tf.keras.layers.GRU(num_units, return_state=True)(enc_input) # input = [batch, timesteps, feature]
    
    # decoder
    dec_input = Input(shape = (None, num_tokens))
    dec_output = tf.keras.layers.GRU(num_units, return_sequences=True)(dec_input, initial_state = enc_state)
    
    # match input and ouput size
    fin_output = Dense(num_tokens, activation='softmax')(dec_output)

    sequence_autoencoder = Model([enc_input, dec_input], fin_output)
    
    encoder = Model(enc_input, enc_output)
    
    return sequence_autoencoder, encoder

In [25]:
# create the model
flatted_feature_size = 32 # number of features per sample (Mel)
num_units = 1
autoenc, enc = rnn_model(flatted_feature_size, num_units)

In [26]:
# generate random input
num_samples = 10
timesteps = 5
# encoder input
X_train = np.random.rand(num_samples, timesteps, flatted_feature_size)

# decoder input, as shifted encoder input
X_train_shifted = np.zeros(X_train.shape)
# loop in timesteps
for sample in range(0, X_train.shape[0]):
    for timestep in range(0, X_train.shape[1] - 1):
        X_train_shifted[sample, timestep + 1, :] = X_train[sample, timestep, :]

# train the model
autoenc.compile(optimizer = "adam", loss = tf.keras.losses.MeanSquaredError(), metrics = ["accuracy"])
autoenc.fit(x = [X_train, X_train_shifted], y = X_train, epochs = 2, batch_size = 2)

Epoch 1/2
10/10 [==============================] - 1s 94ms/sample - loss: 0.3158 - acc: 0.0200
Epoch 2/2
10/10 [==============================] - 0s 2ms/sample - loss: 0.3157 - acc: 0.0200
